<a href="https://colab.research.google.com/github/sergioGarcia91/BucaramangaSeismicNest_ML/blob/main/ML_SismosNido_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, the 10 models with the highest AUC are selected; logistic regression is fitted, and LDA and PCA are applied to evaluate the behavior of the variables and the separability of the classes.

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_vs_lda.html#sphx-glr-auto-examples-decomposition-plot-pca-vs-lda-py

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

# Start

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier # Para la Red Neuronal
from sklearn.linear_model import LogisticRegression
from joblib import dump, load # guardar el modelo
from matplotlib.patches import Rectangle # Para hacer los rectangulos
from matplotlib.collections import PatchCollection # Tambien para los rectangulos
from datetime import datetime, timedelta
from sklearn.metrics import RocCurveDisplay, roc_curve
from sklearn.metrics import roc_auc_score


In [3]:
pathDatos = '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/Catalogos/'
pathSaveFiguras = '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/Figuras_LDA_PCA_RL/'

# Change the font type

In [4]:
import matplotlib as mpl
import matplotlib.font_manager as fm

In [5]:
!wget https://github.com/justrajdeep/fonts/raw/master/Times%20New%20Roman.ttf

--2025-08-24 12:44:26--  https://github.com/justrajdeep/fonts/raw/master/Times%20New%20Roman.ttf
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/justrajdeep/fonts/master/Times%20New%20Roman.ttf [following]
--2025-08-24 12:44:26--  https://raw.githubusercontent.com/justrajdeep/fonts/master/Times%20New%20Roman.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 834452 (815K) [application/octet-stream]
Saving to: ‘Times New Roman.ttf’

Times New Roman.ttf 100%[===================>] 814.89K  3.20MB/s    in 0.2s    

2025-08-24 12:44:27 (3.20 MB/s) - ‘Times New Roman.ttf’ saved [834452/834452]



In [6]:
# Ruta a la fuente personalizada
font_path = 'Times New Roman.ttf'

# Añadir la fuente al administrador de fuentes de Matplotlib
font_prop = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)

# Nombre de la fuente para usar en rcParams
font_name = font_prop.get_name()
font_name

'Times New Roman'

In [7]:
plt.rcParams['font.family'] = font_name

# Load data

In [8]:
# Se carga el catalogo de los eventos de prof mayores a 100 km
df = pd.read_csv(pathDatos+'df_Total_1994_2024.csv')
df['Date-Time'] = pd.to_datetime(df['Date-Time'], yearfirst=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148986 entries, 0 to 148985
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   FECHA                   148986 non-null  object        
 1   HORA_UTC                148986 non-null  object        
 2   LATITUD (grados)        148986 non-null  float64       
 3   LONGITUD (grados)       148986 non-null  float64       
 4   PROFUNDIDAD (Km)        148986 non-null  float64       
 5   MAGNITUD Ml             148986 non-null  float64       
 6   ERROR LATITUD (Km)      148910 non-null  float64       
 7   ERROR LONGITUD (Km)     148910 non-null  float64       
 8   ERROR PROFUNDIDAD (Km)  148910 non-null  float64       
 9   Date-Time               148986 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 11.4+ MB


In [9]:
df_Dias = pd.read_csv(pathDatos+'df_Dias_v3_ml15_2009.csv')
df_Dias['Fecha'] = pd.to_datetime(df_Dias['Dia'], yearfirst=True)
filtro2009 = (df_Dias['Fecha'].dt.year >= 2009).to_numpy()
df_Dias = df_Dias[filtro2009]
df_Dias.reset_index(inplace=True, drop=True)
df_Dias

,Dia,1.5-2.0,2.0-2.5,2.5-3.0,3.0-3.5,3.5-4.0,4.0-4.5,4.5-5.0,5.0-5.5,5.5-,Total dia,Fecha
0,2009-01-01,5,0,1,0,0,0,0,0,0,6,2009-01-01
1,2009-01-02,4,0,2,1,0,0,0,0,0,7,2009-01-02
2,2009-01-03,5,1,0,1,0,0,0,0,0,7,2009-01-03
3,2009-01-04,7,1,2,0,0,0,0,0,0,10,2009-01-04
4,2009-01-05,5,2,0,0,0,0,0,0,0,7,2009-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...
5533,2024-02-25,44,5,1,0,0,0,0,0,0,50,2024-02-25
5534,2024-02-26,31,7,6,1,0,0,0,0,0,45,2024-02-26
5535,2024-02-27,37,5,2,0,0,0,0,0,0,44,2024-02-27
5536,2024-02-28,35,7,4,0,0,0,0,0,0,46,2024-02-28


# Path 10 AUC models

In [10]:
# best 10 AUC
ordenModelos = ['/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v6/2009_30dias_boots_hl_420_210_105_53_27_5_int0_scr0.997.joblib',
                '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v4/2009_30dias_hl_420_5_int0_scr0.998.joblib',
                '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v4/2009_30dias_hl_2000_5_int1_scr1.0.joblib',
                '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v4/2009_6meses_hl_210_5_int0_scr1.0.joblib',
                '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v6/2009_30dias_boots_hl_1500_5_int0_scr1.0.joblib',
                '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v6/2009_30dias_boots_hl_420_5_int0_scr1.0.joblib',
                '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v4/2009_30dias_hl_1500_5_int0_scr1.0.joblib',
                '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v6/2009_6meses_hl_1500_750_375_188_94_5_int0_scr0.996.joblib',
                '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v6/2009_30dias_boots_hl_2000_5_int0_scr0.999.joblib',
                '/content/drive/MyDrive/Manuscritos_Investigacion/ML_SismosNidoBucaramanga/ModelosMLP_Class_v6/2009_6meses_hl_210_105_53_27_13_5_int0_scr1.0.joblib']

# label
labelModel = ['Mod 7',
              'Mod 15',
              'Mod 32',
              'Mod 41',
              'Mod 26',
              'Mod 13',
              'Mod 47',
              'Mod 12',
              'Mod 25',
              'Mod 24']


In [ ]:
inicial_model = 0

for modelo, labelModelo in zip(ordenModelos[inicial_model:], labelModel[inicial_model:]):
  if 'joblib' in modelo:
    if '6meses' in modelo:
      dias_Considerar = 180
      X2023 = np.loadtxt(pathDatos+'2009_6meses_X2022.txt')
      y2023 = np.loadtxt(pathDatos+'2009_6meses_y2022.txt')
      X2022 = np.loadtxt(pathDatos+'2009_6meses_X2021.txt')
      y2022 = np.loadtxt(pathDatos+'2009_6meses_y2021.txt')

    if '12meses' in modelo:
      dias_Considerar = 365
      X2023 = np.loadtxt(pathDatos+'2009_12meses_X2022.txt')
      y2023 = np.loadtxt(pathDatos+'2009_12meses_y2022.txt')
      X2022 = np.loadtxt(pathDatos+'2009_12meses_X2021.txt')
      y2022 = np.loadtxt(pathDatos+'2009_12meses_y2021.txt')

    if '30dias' in modelo:
      dias_Considerar = 30
      X2023 = np.loadtxt(pathDatos+'2009_30dias_X2022.txt')
      y2023 = np.loadtxt(pathDatos+'2009_30dias_y2022.txt')
      X2022 = np.loadtxt(pathDatos+'2009_30dias_X2021.txt')
      y2022 = np.loadtxt(pathDatos+'2009_30dias_y2021.txt')

    print(10*'---')
    print(labelModelo)
    print('Train shape: ', X2022.shape)
    print('Test shape: ', X2023.shape)
    print('Y test sum: ', y2023.sum())

    # labels days
    days_labels = []
    days = int(X2022.shape[1]/7)
    print(days)
    for day in range(days-1, -1, -1):
      for ml in ['2.5-3.0', '3.0-3.5', '3.5-4.0', '4.0-4.5', '4.5-5.0', '5.0-5.5', '>=5.5']:
        label = f'-{day}D Ml[{ml}]'
        days_labels.append(label)
    print(days_labels)
    print(len(days_labels)==X2022.shape[1])


    # Create Logistic Regression model
    min_max_scaler = MinMaxScaler()
    min_max_scaler.fit(X2022)
    X2022_s = min_max_scaler.transform(X2022)
    model_LogReg = LogisticRegression(solver='saga',
                                      verbose=True,
                                      n_jobs=-1) # -1 means using all processors
    model_LogReg.fit(X2022_s, y2022)

    # Comparison of LDA and PCA 2D projection
    pca = PCA(n_components=2)
    X_r = pca.fit(X2022_s).transform(X2022_s)

    lda = LinearDiscriminantAnalysis(n_components=1) # no puede ser 2 ...
    X_r2 = lda.fit(X2022_s, y2022).transform(X2022_s)

    colors = ["tab:blue", "tab:orange"]
    target_names = ['Class 0', 'Class 1']
    class_map = {0: target_names[0], 1: target_names[1]}
    y_kde = pd.Series(y2022, name='Class').map(class_map)
    palette_kde = {target_names[0]: colors[0], target_names[1]: colors[1]}

    fig, ax = plt.subplots(1,2, figsize=(9, 4))

    sns.kdeplot(x=X_r2.ravel(),
                hue=y_kde,
                hue_order=[target_names[0], target_names[1]],
                palette=palette_kde,
                fill=True,
                ax=ax[0],
                color=colors)
    ax[0].grid(ls='--', color='grey', alpha=0.8)
    ax[0].set_xlabel('1D component', fontsize=10)
    ax[0].set_ylabel('Density', fontsize=10)
    ax[0].set_title(f'LDA (1 component) – training data: {labelModelo}', fontsize=10)

    for color, i, target_name in zip(colors, [0, 1], target_names):
        ax[1].scatter(
            X_r[y2022 == i, 0],
            X_r[y2022 == i, 1],
            color=color,
            s = 2,
            alpha=0.8,
            label=target_name
        )
    ax[1].grid(ls='--', color='grey', alpha=0.8)
    ax[1].set_xlabel('PC1', fontsize=10)
    ax[1].set_ylabel('PC2', fontsize=10)
    ax[1].set_title(f'PCA (2 components) – training data: {labelModelo}', fontsize=10)
    ax[1].legend(title='Class', fontsize=10)

    plt.subplots_adjust(wspace=0.2)
    plt.tight_layout()
    plt.savefig((pathSaveFiguras + f'LDA_PCA_{labelModelo}.png'),
                format='png', dpi=1000, bbox_inches = 'tight',pad_inches=0.25)
    plt.show()

    # Coefficients Logistic Regretion
    dict_coef_days = {'coef_LogReg': model_LogReg.coef_[0],
                      'coef_LogReg_abs': np.abs(model_LogReg.coef_[0]),
                      'days_labels': days_labels}
    df_coef_days = pd.DataFrame(dict_coef_days)
    df_coef_days.sort_values(by=['coef_LogReg_abs'],
                             ascending=False,
                             inplace=True)
    df_coef_days.reset_index(inplace=True, drop=True)
    df_coef_days = df_coef_days.iloc[:40, :]
    df_coef_days['Sign'] = np.where(df_coef_days['coef_LogReg'] < 0, 'Negative', 'Positive')

    plt.figure(figsize=(5, 8))
    sns.barplot(df_coef_days,
                x='coef_LogReg_abs',
                y='days_labels',
                hue='Sign')
    plt.title(f'Logistic Regression Coefficient - {labelModelo}', fontsize=12, fontname='Times New Roman')
    plt.xlabel('Absolute coefficient value', fontsize=10, fontname='Times New Roman')
    plt.ylabel('Top 40 features', fontsize=10, fontname='Times New Roman')
    plt.grid(ls='--', color='grey', alpha=0.8)

    plt.savefig((pathSaveFiguras + f'Coefs_{labelModelo}_LogReg_pred.png'),
                format='png', dpi=1000, bbox_inches = 'tight',pad_inches=0.25)

    plt.show()

    # MLPClassifier AUC
    modelo_P = load(modelo)
    n_features = X2022.shape[1]

    # 2023
    predichosModelo = modelo_P.predict(X2023)
    predProb_Modelo = modelo_P.predict_proba(X2023)[:,1]
    auc_score = roc_auc_score(y2023, predProb_Modelo)

    X2023_s = min_max_scaler.transform(X2023)

    predichosModelo_LogReg = model_LogReg.predict(X2023_s)
    predProb_Modelo_LogReg = model_LogReg.predict_proba(X2023_s)[:,1]
    auc_score_LogReg = roc_auc_score(y2023, predProb_Modelo_LogReg)

    predichosModelo_LDA = lda.predict(X2023_s)
    predProb_Modelo_LDA = lda.predict_proba(X2023_s)[:,1]
    auc_score_LDA = roc_auc_score(y2023, predProb_Modelo_LDA)

    auc_score_LogReg = np.round(auc_score_LogReg, 3)
    auc_score_LDA = np.round(auc_score_LDA, 3)
    auc_score = np.round(auc_score, 3)

    # 1994-2023
    #predichosModelo_2022 = modelo_P.predict(X2022)
    #predProb_Modelo_2022 = modelo_P.predict_proba(X2022)[:,1]

    nMin = dias_Considerar - 1
    filtro2023 = (df_Dias['Fecha'].iloc[dias_Considerar-1:-6].dt.year >= 2022).to_numpy()
    fechas2023 = np.arange('2022-01-01', '2024-03-01', dtype='datetime64[D]')
    fechas2023 = fechas2023[:-6]
    # Sacar las fechas inicio fin de los rectangulos
    fechaInicio = []
    fechaFin = []

    for i in range(0, len(y2023)-1):
      if (y2023[i] == 0) & (y2023[i+1] == 1):
        fechaInicio.append(fechas2023[i+1])
      if (y2023[i] == 1) & (y2023[i+1] == 0):
        fechaFin.append(fechas2023[i])
    fechaInicio = np.array(fechaInicio).reshape(-1,1)
    fechaFin = np.array(fechaFin).reshape(-1,1)
    fechasRectangulo = np.concatenate((fechaInicio, fechaFin), axis=1)

    # Plots Log Reg 2023
    fig, ax = plt.subplots(4,1, figsize=(5.5,5.5), sharex=True, gridspec_kw={'height_ratios': [1, 1, 1, 10]})

    # Rectangulos
    for rec in range(0, fechasRectangulo.shape[0]):
      deltaDias = fechasRectangulo[rec,1] - fechasRectangulo[rec,0]
      Xrec = fechasRectangulo[rec,0]
      Yrec = -1
      ax[0].add_patch(Rectangle(xy=(Xrec, Yrec),
                              width=(fechasRectangulo[rec,1] - fechasRectangulo[rec,0]),
                              height=10,
                              fill=True,
                              color ='red',
                              alpha=0.3)
                              )
    predSis_MLPClas = ax[0].scatter(df_Dias['Fecha'].iloc[nMin:-6][filtro2023][predichosModelo >= 1],
                predichosModelo[predichosModelo >= 1]*7,
                s= 1,
                c='b',
                label=f'Predicted: {int(sum(predichosModelo))}')
    #ax[0].legend(loc=1, fontsize=10)
    ax[0].set_ylim(6.5, 7.5)
    #ax[0].set_ylabel('Probabilidad [%]', fontsize=10)
    ax[0].set_yticks([])
    #ax[0].set_xlabel('Fecha - UTC')
    ax[0].grid(ls='--', color='grey')
    ax[0].set_title(f'{labelModelo}: {int(sum(predichosModelo))} predictions [2022-feb2024] - [AUC:{auc_score}]', fontsize=10)


    # Rectangulos
    for rec in range(0, fechasRectangulo.shape[0]):
      deltaDias = fechasRectangulo[rec,1] - fechasRectangulo[rec,0]
      Xrec = fechasRectangulo[rec,0]
      Yrec = -1
      ax[1].add_patch(Rectangle(xy=(Xrec, Yrec),
                              width=(fechasRectangulo[rec,1] - fechasRectangulo[rec,0]),
                              height=10,
                              fill=True,
                              color ='red',
                              alpha=0.3)
                              )
    predSis_MLPClas = ax[1].scatter(df_Dias['Fecha'].iloc[nMin:-6][filtro2023][predichosModelo_LogReg >= 1],
                predichosModelo_LogReg[predichosModelo_LogReg >= 1]*7,
                s= 1,
                c='b',
                label=f'Predicted: {int(sum(predichosModelo_LogReg))}')
    #ax[0].legend(loc=1, fontsize=10)
    ax[1].set_ylim(6.5, 7.5)
    #ax[0].set_ylabel('Probabilidad [%]', fontsize=10)
    ax[1].set_yticks([])
    #ax[0].set_xlabel('Fecha - UTC')
    ax[1].grid(ls='--', color='grey')
    ax[1].set_title(f'LogReg: {int(sum(predichosModelo_LogReg))} predictions [2022-feb2024] - [AUC:{auc_score_LogReg}]', fontsize=10)

    # Rectangulos
    for rec in range(0, fechasRectangulo.shape[0]):
      deltaDias = fechasRectangulo[rec,1] - fechasRectangulo[rec,0]
      Xrec = fechasRectangulo[rec,0]
      Yrec = -1
      ax[2].add_patch(Rectangle(xy=(Xrec, Yrec),
                              width=(fechasRectangulo[rec,1] - fechasRectangulo[rec,0]),
                              height=10,
                              fill=True,
                              color ='red',
                              alpha=0.3)
                              )
    predSis_MLPClas = ax[2].scatter(df_Dias['Fecha'].iloc[nMin:-6][filtro2023][predichosModelo_LDA >= 1],
                predichosModelo_LDA[predichosModelo_LDA >= 1]*7,
                s= 1,
                c='b',
                label=f'Predicted: {int(sum(predichosModelo_LDA))}')
    #ax[0].legend(loc=1, fontsize=10)
    ax[2].set_ylim(6.5, 7.5)
    #ax[0].set_ylabel('Probabilidad [%]', fontsize=10)
    ax[2].set_yticks([])
    #ax[0].set_xlabel('Fecha - UTC')
    ax[2].grid(ls='--', color='grey')
    ax[2].set_title(f'LDA: {int(sum(predichosModelo_LDA))} predictions [2022-feb2024] - [AUC:{auc_score_LDA}]', fontsize=10)

    # Eventos logrados
    event4_5 = df['MAGNITUD Ml'][(df['Date-Time'].dt.year >= 2022).to_numpy()]
    event4_5 = event4_5 >= 4.5
    event4_5 = sum(event4_5)

    ax[3].scatter(df['Date-Time'][(df['Date-Time'].dt.year >= 2022).to_numpy()],
                  df['MAGNITUD Ml'][(df['Date-Time'].dt.year >= 2022).to_numpy()],
                  s=1,
                  label=f'Earthquakes ({event4_5} events Ml$\geq$4.5)',
                  c='tab:blue',
                  alpha=1)

    # Rectangulos
    for rec in range(0, fechasRectangulo.shape[0]):
      deltaDias = fechasRectangulo[rec,1] - fechasRectangulo[rec,0]
      Xrec = fechasRectangulo[rec,0]
      Yrec = -1
      ax[3].add_patch(Rectangle(xy=(Xrec, Yrec),
                              width=(fechasRectangulo[rec,1] - fechasRectangulo[rec,0]),
                              height=8,
                              fill=True,
                              color ='red',
                              alpha=0.3)
                              )

    left, right = plt.xlim()
    ax[3].hlines(y= 4.5,
                xmin=left,
                xmax=right,
                ls='--',
                lw=1,
                color='r',)
    ax[3].set_ylim(-0.5, 7.5)
    ax[3].set_ylabel('Ml', fontsize=10)
    ax[3].set_xlabel('Date - UTC', fontsize=10)
    ax[3].grid(ls='--', color='grey')
    ax[3].set_yticks(np.arange(0, 8, 0.5))
    ax[3].set_ylim(0,6)
    ax[3].legend(loc=8, fontsize=10)

    plt.subplots_adjust(hspace=0.3)
    plt.xticks(np.arange(np.datetime64("2022-01-01"), np.datetime64("2024-03-01"),  np.timedelta64(4, 'M'), dtype='datetime64[M]'),
               labels=np.arange(np.datetime64("2022-01-01"), np.datetime64("2024-03-01"),  np.timedelta64(4, 'M'), dtype='datetime64[M]'))

    plt.xlim(np.datetime64('2022-01-01'), np.datetime64('2024-03-01'))
    plt.tick_params(labelsize=10)

    #plt.tight_layout()
    plt.savefig((pathSaveFiguras + f'2022_{labelModelo}_MLPClass_LogReg_pred.png'),
                format='png', dpi=1000, bbox_inches = 'tight',pad_inches=0.25)

    plt.show()

    print(10*'---')
    print(10*'---')
    print('\n') #

# End